In [1]:
import pandas as pd
import os

def procesar_archivos_excel(base_path, meses, hojas_interes):
    for mes in meses:
        file_path = os.path.join(base_path, mes, f'DATOS EN FORMATO REUTILIZABLE {mes.upper()}.xlsx')
        if not os.path.exists(file_path):
            print(f"El archivo {file_path} no existe.")
            continue
        
        # Crear un diccionario para almacenar los DataFrames con el nombre de cada hoja
        dfs = {}
        
        # Leer cada hoja de interés y almacenarla en el diccionario
        for hoja in hojas_interes:
            try:
                df = pd.read_excel(file_path, sheet_name=hoja, header=2)  # Usar header=2 para que tome los títulos correctos
                dfs[hoja] = df
            except Exception as e:
                print(f"Error al leer la hoja {hoja} del archivo {file_path}: {e}")
        
        # Guardar cada DataFrame en un archivo CSV
        for hoja, df in dfs.items():
            output_path = os.path.join(base_path, mes, f"{hoja.lower().replace(' ', '_')}.csv")
            df.to_csv(output_path, index=False)
            print(f"Archivo guardado: {output_path}")

# Ruta base y meses a procesar
base_path = 'C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/'
meses = [
    'Junio 2024', 'Mayo 2024', 'Abril 2024', 'Marzo 2024', 'Febrero 2024', 'Enero 2024',
    'Diciembre 2023', 'Noviembre 2023', 'Octubre 2023',
    'Septiembre 2023', 'Agosto 2023', 'Julio 2023',
    'Junio 2023', 'Mayo 2023', 'Abril 2023', 'Marzo 2023',
    'Febrero 2023', 'Enero 2023'
]
hojas_interes = ['SEGURIDAD', 'PERS. DETENIDAS X DISTRITOS', 'ACCIDENTES', 'EXPEDIENTES OAC', 'VENTA AMBULANTE', 'AA.TT.']

# Llamar a la función
procesar_archivos_excel(base_path, meses, hojas_interes)

El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\DATOS EN FORMATO REUTILIZABLE JUNIO 2024.xlsx no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Mayo 2024\DATOS EN FORMATO REUTILIZABLE MAYO 2024.xlsx no existe.
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\seguridad.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\pers._detenidas_x_distritos.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\accidentes.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\expedientes_oac.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\venta_ambulante.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\aa.tt..csv
Archivo guardado: C:/Users/Lenovo/Docu

## UNIR DATASET INDICES DE CRIMINALIDAD.

In [2]:
def unir_archivos_csv(base_path, meses, archivos_csv):
    dfs_meses = {}

    # Recorrer cada carpeta de mes y leer los archivos CSV
    for mes in meses:
        carpeta = os.path.join(base_path, mes)
        dfs = {}
        for archivo in archivos_csv:
            ruta_archivo = os.path.join(carpeta, archivo)
            if os.path.exists(ruta_archivo):
                df = pd.read_csv(ruta_archivo)
                if 'COMISARÍAS' in df.columns:
                    df.rename(columns={'COMISARÍAS': 'DISTRITOS'}, inplace=True)
                dfs[archivo] = df
            else:
                print(f"El archivo {ruta_archivo} no existe.")
        
        # Verificar que todos los DataFrames tienen la columna 'DISTRITOS'
        for archivo, df in dfs.items():
            if 'DISTRITOS' not in df.columns:
                print(f"El archivo {archivo} no contiene la columna 'DISTRITOS'. Las columnas son: {df.columns}")

        # Inicializar el DataFrame final con el primer DataFrame de la lista
        if archivos_csv[0] in dfs:
            df_ind_criminalidad = dfs[archivos_csv[0]]
        
            # Realizar un outer join con los demás DataFrames basado en la columna 'DISTRITOS'
            for archivo in archivos_csv[1:]:
                if archivo in dfs and 'DISTRITOS' in dfs[archivo].columns:
                    df_ind_criminalidad = pd.merge(df_ind_criminalidad, dfs[archivo], on='DISTRITOS', how='outer')
                else:
                    print(f"Omitiendo el archivo {archivo} porque no tiene la columna 'DISTRITOS'.")
            
            # Guardar el DataFrame resultante de cada mes en el diccionario
            dfs_meses[mes] = df_ind_criminalidad
        else:
            print(f"No se encontró el archivo {archivos_csv[0]} en el mes {mes}.")

    # Unir los DataFrames de todos los meses
    df_final = pd.concat(dfs_meses.values(), ignore_index=True)
    
    return df_final

# Definir la lista de archivos CSV
archivos_csv = [
    'aa.tt..csv', 'accidentes.csv', 'expedientes_oac.csv',
    'pers._detenidas_x_distritos.csv', 'seguridad.csv', 'venta_ambulante.csv'
]

# Llamar a la función y obtener el DataFrame final
df_final = unir_archivos_csv(base_path, meses, archivos_csv)

# Guardar el DataFrame final en un archivo CSV
df_final.to_csv(os.path.join(base_path, 'df_ind_criminalidad_Madrid_2023_2024.csv'), index=False)
print("Archivo final guardado: df_ind_criminalidad_Madrid_2023_2024.csv")

El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\aa.tt..csv no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\accidentes.csv no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\expedientes_oac.csv no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\pers._detenidas_x_distritos.csv no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\seguridad.csv no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\venta_ambulante.csv no existe.
No se encontró el archivo aa.tt..csv en el mes Junio 2024.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Mayo 2024\aa.tt..csv no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Mayo 2024\accidentes.csv no existe.
El archivo C:/Users/L

## EJECUTABLE.

In [3]:
# Convertir archivos Excel a CSV
procesar_archivos_excel(base_path, meses, hojas_interes)

# Unir archivos CSV en un único DataFrame
df_final = unir_archivos_csv(base_path, meses, archivos_csv)

# Guardar el DataFrame final en un archivo CSV
df_final.to_csv(os.path.join(base_path, 'df_ind_criminalidad_Madrid_2023_2024.csv'), index=False)
print("Archivo final guardado: df_ind_criminalidad_Madrid_2023_2024.csv")

El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Junio 2024\DATOS EN FORMATO REUTILIZABLE JUNIO 2024.xlsx no existe.
El archivo C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Mayo 2024\DATOS EN FORMATO REUTILIZABLE MAYO 2024.xlsx no existe.
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\seguridad.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\pers._detenidas_x_distritos.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\accidentes.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\expedientes_oac.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\venta_ambulante.csv
Archivo guardado: C:/Users/Lenovo/Documents/Master BD/TFM/IND. CRIMINALIDAD/Madrid/Abril 2024\aa.tt..csv
Archivo guardado: C:/Users/Lenovo/Docu

In [4]:
df_final.head(20)


,DISTRITOS,EXPEDIENTES INSTRUIDOS,CONTACTOS CON CENTROS ESCOLARES,CON VICTIMAS,SIN VICTIMAS,EXPEDIENTES,PERSONAS DETENIDAS E INVESTIGADAS,RELACIONADAS CON LAS PERSONAS,RELACIONADAS CON EL PATRIMONIO,POR TENENCIA DE ARMAS,POR TENENCIA DE DROGAS,POR CONSUMO DE DROGAS,PROPIEDAD INTELECTUAL E INDUSTRIAL,INFRACCIONES ALIMENTARIAS
0,ARGANZUELA,0.0,9.0,28.0,62.0,18.0,28.0,6.0,6.0,1.0,20.0,3.0,11.0,4.0
1,BARAJAS,0.0,31.0,9.0,23.0,3.0,11.0,1.0,10.0,1.0,48.0,1.0,0.0,37.0
2,CARABANCHEL,13.0,22.0,46.0,68.0,59.0,69.0,25.0,18.0,6.0,29.0,0.0,14.0,127.0
3,CENTRO,13.0,11.0,60.0,58.0,57.0,149.0,25.0,87.0,4.0,123.0,9.0,119.0,25.0
4,CHAMARTÍN,13.0,37.0,72.0,81.0,12.0,21.0,6.0,12.0,0.0,56.0,16.0,28.0,28.0
5,CHAMBERÍ,2.0,8.0,47.0,44.0,14.0,15.0,4.0,0.0,0.0,27.0,3.0,1.0,3.0
6,CIUDAD LINEAL,2.0,0.0,43.0,47.0,37.0,39.0,12.0,27.0,6.0,21.0,1.0,4.0,7.0
7,FUENCARRAL - EL PARDO,2.0,52.0,39.0,62.0,27.0,44.0,9.0,8.0,0.0,30.0,0.0,1.0,4.0
8,HORTALEZA,12.0,50.0,29.0,43.0,16.0,28.0,5.0,3.0,1.0,14.0,1.0,2.0,2.0
9,LATINA,7.0,7.0,31.0,47.0,40.0,57.0,7.0,8.0,4.0,32.0,3.0,1.0,29.0
